In [1]:
import networkx as nx

def extract_quotient_graph_info(Q):
    # 1. Qの各ノードの部分グラフを構成するGのノードを順番に配列Aに格納
    A = []
    for _, data in sorted(Q.nodes(data=True)):
        A.extend(data['members'])
    
    # 2. Aの要素iがbaseであるかadditionalであるかを示す配列
    node_types = []
    for _, data in sorted(Q.nodes(data=True)):
        node_types.extend([data['type']] * len(data['members']))
    
    # 3. Aの要素iがQの何番のノードに属しているかを示す配列
    node_assignments = []
    for q_node, data in sorted(Q.nodes(data=True)):
        node_assignments.extend([q_node] * len(data['members']))
    
    return A, node_types, node_assignments

def custom_quotient_graph(G, base_partition, additional_partition):
    Q = nx.Graph()
    
    # ノードの作成 (属性1: 基本パーティション)
    for i, part in enumerate(base_partition):
        Q.add_node(i, members=list(part), type='base')
    
    # ノードの作成 (属性2: 追加パーティション)
    for i, part in enumerate(additional_partition, start=len(base_partition)):
        Q.add_node(i, members=list(part), type='additional')
    
    # エッジの作成
    all_partitions = base_partition + additional_partition
    for i, part1 in enumerate(all_partitions):
        for j, part2 in enumerate(all_partitions):
            if i < j:  # 重複を避けるため
                connecting_edges = [(n1, n2) for n1 in part1 for n2 in part2 if G.has_edge(n1, n2)]
                if connecting_edges:
                    if Q.nodes[i]['type'] == Q.nodes[j]['type'] == 'base':
                        Q.add_edge(i, j, connecting_edges=connecting_edges, edge_type='base-base')
                    elif Q.nodes[i]['type'] == Q.nodes[j]['type'] == 'additional':
                        Q.add_edge(i, j, connecting_edges=connecting_edges, edge_type='additional-additional')
                    else:
                        Q.add_edge(i, j, connecting_edges=connecting_edges, edge_type='base-additional')
    
    return Q

# 使用例と動作確認
def print_extracted_info(A, node_types, node_assignments):
    print("Extracted Information:")
    print("A (Nodes of G in order):", A)
    print("Node Types:", node_types)
    print("Node Assignments:", node_assignments)
    
    print("\nDetailed view:")
    for i, (node, node_type, assignment) in enumerate(zip(A, node_types, node_assignments)):
        print(f"Index {i}: Node {node}, Type: {node_type}, Assigned to Q-node: {assignment}")

# テスト用のクオーシェントグラフの作成 (前回の例を使用)
G = nx.Graph()
G.add_edges_from([(1,2), (2,3), (3,4), (4,5), (5,6), (6,1), (3,6)])

base_partition = [[1, 2, 3], [4, 5, 6]]
additional_partition = [[2, 3, 4], [5, 6, 1]]

Q = custom_quotient_graph(G, base_partition, additional_partition)

# 新しい関数を使用して情報を抽出
A, node_types, node_assignments = extract_quotient_graph_info(Q)

# 結果の表示
print_extracted_info(A, node_types, node_assignments)

Extracted Information:
A (Nodes of G in order): [1, 2, 3, 4, 5, 6, 2, 3, 4, 5, 6, 1]
Node Types: ['base', 'base', 'base', 'base', 'base', 'base', 'additional', 'additional', 'additional', 'additional', 'additional', 'additional']
Node Assignments: [0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3]

Detailed view:
Index 0: Node 1, Type: base, Assigned to Q-node: 0
Index 1: Node 2, Type: base, Assigned to Q-node: 0
Index 2: Node 3, Type: base, Assigned to Q-node: 0
Index 3: Node 4, Type: base, Assigned to Q-node: 1
Index 4: Node 5, Type: base, Assigned to Q-node: 1
Index 5: Node 6, Type: base, Assigned to Q-node: 1
Index 6: Node 2, Type: additional, Assigned to Q-node: 2
Index 7: Node 3, Type: additional, Assigned to Q-node: 2
Index 8: Node 4, Type: additional, Assigned to Q-node: 2
Index 9: Node 5, Type: additional, Assigned to Q-node: 3
Index 10: Node 6, Type: additional, Assigned to Q-node: 3
Index 11: Node 1, Type: additional, Assigned to Q-node: 3
